## Generate fill in the blanks from any content

In [1]:
# Installing from https://github.com/boudinfl/pke library for Python Keyword extraction
# We use a fixed commit as the later changes might break the code. If it was on pip we would have used exact version number.

!pip install --quiet git+https://github.com/boudinfl/pke.git@5af1f817e0211c33ac3f90e1e86bb5c1283448e8
!pip install --quiet flashtext==2.7

     |████████████████████████████████| 235 kB 4.7 MB/s 


In [2]:
!pip install spacy==2.2.4
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.6 MB 5.3 MB/s 
     |████████████████████████████████| 2.2 MB 41.0 MB/s 
     |████████████████████████████████| 211 kB 37.3 MB/s 
     |████████████████████████████████| 3.7 MB 37.0 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.9
    Uninstalling blis-0.7.9:
      Successfully uninstalled blis-0.7.9
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: sp

In [3]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /usr/local/lib/python3.8/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-glibc2.27
Python version   3.8.15                        
Models                                         



In [4]:
import textwrap
text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""

wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list: 
  print(element) 



There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic
Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other
at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s
surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental
crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This
makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate
is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pu

In [5]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

sentences = tokenize_sentences(text)
print (sentences)

['There is a lot of volcanic activity at divergent plate boundaries in the oceans.', 'For example, many undersea volcanoes are found along the Mid-Atlantic Ridge.', 'This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean.', 'As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust.', 'Molten rock, called magma, erupts through these cracks onto Earth’s surface.', 'At the surface, the molten rock is called lava.', 'It cools and hardens, forming rock.', 'Divergent plate boundaries also occur in the continental crust.', 'Volcanoes form at these boundaries, but less often than in ocean crust.', 'That’s because continental crust is thicker than oceanic crust.', 'This makes it more difficult for molten rock to push up through the crust.', 'Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone.', 'The leadi

In [7]:
import spacy
def get_noun_adj_verb(text):
    out=[]
    try:
        nlp =spacy.load('en_core_web_sm')
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=text,language='en',spacy_model=nlp)
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        # extractor.candidate_selection(pos=pos, stoplist=stoplist)
        extractor.candidate_selection(pos=pos)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=30)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

noun_verbs_adj = get_noun_adj_verb(text)
print ("keywords: ",noun_verbs_adj)

keywords:  ['oceans', 'undersea volcanoes', 'plate boundaries', 'plate', 'rock', 'crust', 'example', 'cracks', 'activity', 'surface', 'subduction zone', 'volcanoes', 'volcanoes form', 'lot', 'edge', 'north-south', 'earth', 'middle', 'hardens', 'lava', 'fissures', 'convergent plate boundaries', 'line', 'boundaries', 'arc', 'plate melts', 'mantle', 'ocean crust', 'rim']


In [8]:
from pprint import pprint
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

keyword_sentence_mapping_noun_verbs_adj = get_sentences_for_keyword(noun_verbs_adj, sentences)
pprint (keyword_sentence_mapping_noun_verbs_adj)

{'activity': ['There is a lot of volcanic activity at divergent plate '
              'boundaries in the oceans.'],
 'arc': ['When a line of volcanoes forms along a subduction zone, they make up '
         'a volcanic arc.'],
 'boundaries': ['Volcanoes form at these boundaries, but less often than in '
                'ocean crust.'],
 'convergent plate boundaries': ['Many volcanoes form along convergent plate '
                                 'boundaries where one tectonic plate is '
                                 'pulled down beneath another at a subduction '
                                 'zone.'],
 'cracks': ['As tectonic plates pull away from each other at a divergent plate '
            'boundary, they create deep fissures, or cracks, in the crust.',
            'Molten rock, called magma, erupts through these cracks onto '
            'Earth’s surface.'],
 'crust': ['As tectonic plates pull away from each other at a divergent plate '
           'boundary, they create deep f

In [9]:

def get_fill_in_the_blanks(sentence_mapping):
    out={"title":"Fill in the blanks for these sentences with matching words at the top"}
    blank_sentences = []
    processed = []
    keys=[]
    for key in sentence_mapping:
        if len(sentence_mapping[key])>0:
            sent = sentence_mapping[key][0]
            # Compile a regular expression pattern into a regular expression object, which can be used for matching and other methods
            insensitive_sent = re.compile(re.escape(key), re.IGNORECASE)
            no_of_replacements =  len(re.findall(re.escape(key),sent,re.IGNORECASE))
            line = insensitive_sent.sub(' _________ ', sent)
            if (sentence_mapping[key][0] not in processed) and no_of_replacements<2:
                blank_sentences.append(line)
                processed.append(sentence_mapping[key][0])
                keys.append(key)
    out["sentences"]=blank_sentences[:10]
    out["keys"]=keys[:10]
    return out


fill_in_the_blanks = get_fill_in_the_blanks(keyword_sentence_mapping_noun_verbs_adj)
pprint(fill_in_the_blanks)

{'keys': ['oceans',
          'undersea volcanoes',
          'rock',
          'crust',
          'subduction zone',
          'volcanoes',
          'north-south',
          'hardens',
          'lava',
          'line'],
 'sentences': ['There is a lot of volcanic activity at divergent plate '
               'boundaries in the  _________ .',
               'For example, many  _________  are found along the Mid-Atlantic '
               'Ridge.',
               'Molten  _________ , called magma, erupts through these cracks '
               'onto Earth’s surface.',
               'As tectonic plates pull away from each other at a divergent '
               'plate boundary, they create deep fissures, or cracks, in the  '
               '_________ .',
               'Many volcanoes form along convergent plate boundaries where '
               'one tectonic plate is pulled down beneath another at a  '
               '_________ .',
               'The leading edge of the plate melts as it 

In [10]:
from IPython.core.display import display, HTML
import xml.etree.ElementTree as et
import random

root = et.Element("div")

heading = et.Element("h2")
heading.text = fill_in_the_blanks['title']

keywords = et.Element("ul")
keywords.set('style', 'color:blue;')

all_keys = fill_in_the_blanks['keys']
random.shuffle(all_keys)
for blank in all_keys:
  child=et.Element("li")
  child.text = blank
  keywords.append(child)

sentences = et.Element("ol")
sentences.set('style', 'color:brown;')
for sentence in fill_in_the_blanks['sentences']:
  child=et.Element("li")
  child.text = sentence
  sentences.append(child)
  sentences.append(et.Element("br"))

heading_content = et.Element("h4")

root.append(heading)
heading_content.append(keywords)
heading_content.append(sentences)
root.append(heading_content)

xmlstr = et.tostring(root)
xmlstr = xmlstr.decode("utf-8") 
display(HTML(xmlstr))

In [11]:
from xml.dom import minidom
prettyxmlstr = minidom.parseString(et.tostring(root)).toprettyxml(indent="   ")
print (prettyxmlstr)

<?xml version="1.0" ?>
<div>
   <h2>Fill in the blanks for these sentences with matching words at the top</h2>
   <h4>
      <ul style="color:blue;">
         <li>hardens</li>
         <li>line</li>
         <li>subduction zone</li>
         <li>oceans</li>
         <li>north-south</li>
         <li>volcanoes</li>
         <li>rock</li>
         <li>lava</li>
         <li>crust</li>
         <li>undersea volcanoes</li>
      </ul>
      <ol style="color:brown;">
         <li>There is a lot of volcanic activity at divergent plate boundaries in the  _________ .</li>
         <br/>
         <li>For example, many  _________  are found along the Mid-Atlantic Ridge.</li>
         <br/>
         <li>Molten  _________ , called magma, erupts through these cracks onto Earth’s surface.</li>
         <br/>
         <li>As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the  _________ .</li>
         <br/>
         <li>Many volcanoes